In [1]:
import pandas as pd
import time
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
import traceback
import os.path
import csv
from csv import writer
import urllib.request

In [14]:
def initializeDriver():
    """
    Initializes the web browser driver.

    Returns
    -------
    driver
    """
    
    chrome_path = "chromedriver"
    
    options = webdriver.ChromeOptions()
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument('--remote-debugging-port=9222')
    
    driver=webdriver.Chrome(chrome_path,options=options)
    
    return driver

def cleanText(text):
    """
    Replaces non-alphanumeric characters in text with a space
    """
    import re
    text = re.sub('[^0-9a-zA-Z]+', ' ', text)
    text = text.strip()
    
    return text

def scrape_article_text(driver):
                    
    try:
        #text = driver.find_element_by_xpath('/html/body/div[1]/div/div[4]/div[1]/article/div[1]').text

        params =  driver.find_elements_by_xpath('/html/body/div[1]/div/div[4]/div[1]/article/p')

        for p in params:
            print(p.text)

    except Exception as ex:
        try:
            # text = driver.find_element_by_xpath("/html/body/div[1]/div/div[2]/div/div[1]/article/div/div/div/div").text
            params =  driver.find_elements_by_xpath('/html/body/div[1]/div/div[2]/div/div[1]/article/p')

            for p in params:
                print(p.text)

        except Exception as ex:
            print(str(ex))

def scrape_timestamp(driver):
    
    timestamp = None
    
    try:
        timestamp = driver.find_element_by_xpath("/html/head/meta[@name='iso-8601-publish-date']").get_attribute('content')

    except Exception as ex:
            print(str(ex))
    
    return timestamp

def scrape_page(link, driver):
    
    driver.get(link)
    
    article_list = driver.find_elements_by_class_name("storyItem__192ee8af")
    
    hrefs = []
    
    data_list = []
    
    for article in article_list:
        try:
            title = cleanText(article.find_element_by_xpath('./div/a[1]').text)
            href = article.find_element_by_xpath('./div/a[1]').get_attribute('href')
            desc = cleanText(article.find_element_by_xpath('./div/a[2]').text)
            date = article.find_element_by_xpath('./div/div[3]').text
            category = article.find_element_by_xpath('./div/div[1]').text
            
            hrefs.append(href)
            
            data_list.append([href,title,desc,date,category])
            
        except Exception as ex:
            print(str(ex))
            
    df = pd.DataFrame(data_list)
    df.columns = ['href','title','desc','date','category']
    
    df['timestamp'] = pd.Series(index=df.index, dtype='object')
    
    i = 0
    for href in hrefs:
        i+=1
        print(str(i))
        driver.get(href)
        
        idx = df.index[df['href'] == href].tolist()[0]
        
        timestamp = scrape_timestamp(driver)
        df['timestamp'].iloc[idx] = timestamp
        
    return df

In [17]:
driver = initializeDriver()

for i in range(1,1000):
    print("Page: "+str(i))
    link ='https://www.bloomberg.com/search?query=Japanese&sort=time:desc&page='+str(i)
    df = scrape_page(link, driver)
    print(df.head(10))
    df.to_csv("scraped_news_bloomberg.csv",mode='a', header=False, index=False)

Page: 1


ValueError: Length mismatch: Expected axis has 0 elements, new values have 5 elements